# Interactively Writing Web Applications

Reimagining the world of web development with Klein and Jupyter

**Moshe Zadka**

**https://cobordism.com**

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## The The Editor...And Beyond
* Interactive
* Development
* Environment

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Auto-Reloading: A Non-Solution to a Non-Problem

* Complicated and fragile
* Downtime
* Is it reloaded? But how about now?

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## What If I Told You?

I will write a web application during this talk.
At each step,
it will be usable.

There will be no reloading...just progress.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Twisted, Meet Jupyter

* Klein on Twisted
* Twisted on Asyncioreactor
* Jupyter on Asyncio event loop

In [1]:
from twisted.internet import asyncioreactor
asyncioreactor.install()

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## The Imports

In Lisp, every file ends with `))))))))))`.

In Python, every file starts with `import import import`

In [21]:
from twisted.internet import endpoints
from twisted.web import server
from twisted.internet import reactor
from klein import app, resource, route, Klein
import json
import treq

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## When Endpoint Met Resource

* Klein wants to serve web requests, but doesn't know how to network
* Endpoints know how to network, but not how to send content

The romantic comedy coming soon to a conference near you!

In [3]:
endpoint = endpoints.serverFromString(reactor, "tcp:8000")
site = server.Site(resource())
endpoint.listen(site)

<Deferred at 0x7f937a498f10 current result: <<class 'twisted.internet.tcp.Port'> of <class 'twisted.web.server.Site'> on 8000>>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Let's Check It Out

In [4]:
import webbrowser
webbrowser.open("http://localhost:8000")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## If You Don't Succeed At First, Redefine Success

We are not in the *web development business*.

We are in the *incremental web development business*.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

In [7]:
@route("/")
def welcome(request):
    return "<html>Hello world</html>"
webbrowser.open("http://localhost:8000")

True

## No Reloading -- No Race Conditions

In [8]:
@route('/about')
def about(request):
    return 'This is the about page'
@route("/")
def welcome(request):
    return '<html>Hello world. See <a href="/about">About</a></html>'
webbrowser.open("http://localhost:8000")

True

## Modifying Routes

Added a link to an existing route -- and it changed!

## Adding a backend

In [11]:
backend = Klein()

In [16]:
@backend.route("/negate/<int:number>")
def negate(request, number):
    return json.dumps(-number)

In [13]:
endpoint = endpoints.serverFromString(reactor, "tcp:8001")
site = server.Site(backend.resource())
endpoint.listen(site)

<Deferred at 0x7f937a3c2510 current result: <<class 'twisted.internet.tcp.Port'> of <class 'twisted.web.server.Site'> on 8001>>


Traceback (most recent call last):
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/twisted/web/server.py", line 302, in render
    body = resrc.render(self)
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/klein/_resource.py", line 204, in render
    d = defer.maybeDeferred(_execute)
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/twisted/internet/defer.py", line 151, in maybeDeferred
    result = f(*args, **kw)
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/klein/_resource.py", line 198, in _execute
    **kwargs)
--- <exception caught here> ---
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/twisted/internet/defer.py", line 151, in maybeDeferred
    result = f(*args, **kw)
  File "/home/moshez/src/penguin-bit-by-bit/build/pbbb/lib/python3.7/site-packages/klein/_app.py", line 134, in execute_endpoint
    return e

In [23]:
d = treq.get("http://localhost:8001/negate/5")
d.addCallback(lambda x: x.json())
d.addCallback(print)

In [24]:
@backend.route("/mult/<int:number>/<int:number2>")
def negate(request, number, number2):
    return json.dumps(number*number2)